In [21]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
import os
import cv2
import numpy as np

# Define the model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(225, 225, 3)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(5, activation='softmax')
])

In [26]:
images = []
labels = []

# Load the dataset 
dataset_dir = r''

img_width, img_height = 225, 225
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png')):
            label = os.path.basename(root)
            image_path = os.path.join(root, file)
            
            # Read and preprocess the image
            image = cv2.imread(image_path)
            image = cv2.resize(image, (img_width, img_height))
            image = image / 255.0

            images.append(image)
            labels.append(label)

In [27]:
images = np.array(images)
labels = np.array(labels)
print(images)
print(labels)

[[[[0.21960784 0.29019608 0.33333333]
   [0.28235294 0.35294118 0.39607843]
   [0.3372549  0.40784314 0.45098039]
   ...
   [0.1254902  0.14117647 0.14509804]
   [0.13333333 0.14901961 0.15294118]
   [0.0745098  0.09019608 0.09411765]]

  [[0.33333333 0.40392157 0.44705882]
   [0.36862745 0.43921569 0.48235294]
   [0.38039216 0.45098039 0.49411765]
   ...
   [0.0745098  0.09019608 0.09411765]
   [0.06666667 0.08235294 0.08627451]
   [0.0745098  0.09019608 0.09411765]]

  [[0.39215686 0.4627451  0.50588235]
   [0.41176471 0.48235294 0.5254902 ]
   [0.38039216 0.45098039 0.49411765]
   ...
   [0.06666667 0.08235294 0.08627451]
   [0.05098039 0.06666667 0.07058824]
   [0.0627451  0.07843137 0.08235294]]

  ...

  [[0.18823529 0.23921569 0.27058824]
   [0.18431373 0.23529412 0.26666667]
   [0.19607843 0.24705882 0.27843137]
   ...
   [0.1254902  0.1254902  0.1254902 ]
   [0.27843137 0.27843137 0.27843137]
   [0.30980392 0.32156863 0.31372549]]

  [[0.18431373 0.23529412 0.26666667]
   [0.1

In [28]:
# for future use

import cv2
import numpy as np

def preprocess_image(images, image_size=(225, 225)):
    # Resize the image to the desired input size
    images = cv2.resize(images, image_size)
    images = cv2.cvtColor(images, cv2.COLOR_BGR2GRAY)
    
    # Normalizing the pixel values to the range [0, 1]
    images = images.astype('float32') / 255.0
    images = np.expand_dims(images, axis=0)
    
    return images

In [29]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [32]:
# Encode labels for training, validation, and test sets using LabelEncoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Fixing the label
from tensorflow.keras.utils import to_categorical
y_train_encoded = to_categorical(y_train, num_classes=5)
y_test_encoded = to_categorical(y_test, num_classes=5)

In [31]:
print(y_train_encoded,y_test_encoded)
print(X_train,X_test)

[[0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]] [[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]
[[[[0.26666667 0.30196078 0.3372549 ]
   [0.34117647 0.37647059 0.41176471]
   [0.11372549 0.15686275 0.18823529]
   ...
   [0.34901961 0.42352941 0.45098039]
   [0.36078431 0.43529412 0.4627451 ]
   [0.23921569 0.31372549 0.34117647]]

  [[0.31764706 0.35294118 0.38823529]
   [0.34509804 0.38039216 0.41568627]
   [0.1254902  0.16862745 0.2       ]
   ...
   [0.31764706 0.39215686 0.41960784]
   [0.35294118 0.42745098 0.45490196]
   [0.29019608 0.36470588 0.39215686]]

  [[0.32941176 0.36470588 0.4       ]
   [0.27843137 0.31372549 0.34901961]
   [0.09803922 0.14117647 0.17254902]
   ...
   [0.21568627 0.29019608 0.31764706]
   [0.26666667 0.34117647 0.36862745]
   [0.26666667 0.34117647 0.36862745]]

  ...

  [[0.27843137 0.34901961 0.35294118]
   [0.34117647 0

In [33]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [55]:
# Train the Model
model.fit(X_train, y_train_encoded, validation_data=(X_test, y_test_encoded), epochs=20)

Epoch 1/20
59/59 [==============================] - 61s 1s/step - loss: 3.1139 - accuracy: 0.2690 - val_loss: 1.5595 - val_accuracy: 0.3277
Epoch 2/20
59/59 [==============================] - 46s 773ms/step - loss: 1.4731 - accuracy: 0.3601 - val_loss: 1.4607 - val_accuracy: 0.3617
Epoch 3/20
59/59 [==============================] - 44s 752ms/step - loss: 1.2683 - accuracy: 0.5157 - val_loss: 1.5457 - val_accuracy: 0.3511
Epoch 4/20
59/59 [==============================] - 45s 760ms/step - loss: 0.8814 - accuracy: 0.6947 - val_loss: 1.7327 - val_accuracy: 0.3702
Epoch 5/20
59/59 [==============================] - 46s 787ms/step - loss: 0.5306 - accuracy: 0.8300 - val_loss: 2.2223 - val_accuracy: 0.3511
Epoch 6/20
59/59 [==============================] - 60s 1s/step - loss: 0.3493 - accuracy: 0.9084 - val_loss: 2.2435 - val_accuracy: 0.3617
Epoch 7/20
59/59 [==============================] - 45s 757ms/step - loss: 0.2785 - accuracy: 0.9265 - val_loss: 2.4992 - val_accuracy: 0.3979
Epoch

In [57]:
model.save('trained_model.h5')

In [59]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# Decided to not use this lib
# from tensorflow.keras.applications.vgg16 import preprocess_input 

# Path to the image to be predicted 
img_path = (r'')

img = load_img(img_path, target_size=(225, 225))
img_array = img_to_array(img)
# img_array = preprocess_input(img_array)
img_array = img_array.reshape((1, 225, 225, 3))

predictions = model.predict(img_array)
print(predictions)


1/1 [==============================] - 0s 83ms/step
[[0. 0. 1. 0. 0.]]


In [60]:
# Test for a preprocessing method to highlight edges (not used) 

import cv2
# Load the image
image = cv2.imread(r'')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur and bilateral filter to reduce noise
blurred = cv2.GaussianBlur(gray, (13, 13), 0)
blurred = cv2.bilateralFilter(blurred, d=9, sigmaColor=75, sigmaSpace=75)

# Canny edge detection to detect edges
edges = cv2.Canny(blurred, 50, 150)

# Find contours in the edge detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contours over the original image
crack_image = image.copy()
cv2.drawContours(crack_image, contours, -1, (0, 255, 0), 2)  # Green color for contours

# Show the original image with detected cracks
# cv2.imshow('blur', blurred)
cv2.imshow('Crack Detection', crack_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [61]:
# Hard coding for the GSI value to be determined

val = input("Enter your value: ")
if predictions[0][0] == 1:
    if val == 'very good':
        print('100-77')
    elif val == 'good':
        print('89-67')
    elif val == 'fair':
        print('80-56')
    elif val == 'poor':
        print('NA')
    elif val == 'very poor':
        print('NA')
elif predictions[0][1] == 1:
    if val == 'very good':
        print('86-65')
    elif val == 'good':
        print('77-55')
    elif val == 'fair':
        print('66-45')
    elif val == 'poor':
        print('55-35')
    elif val == 'very poor':
        print('43-26')
elif predictions[0][2] == 1:
    if val == 'very good':
        print('74-55')
    elif val == 'good':
        print('65-45')
    elif val == 'fair':
        print('54-36')
    elif val == 'poor':
        print('44-27')
    elif val == 'very poor':
        print('34-18')
elif predictions[0][3] == 1:
    if val == 'very good':
        print('63-45')
    elif val == 'good':
        print('54-38')
    elif val == 'fair':
        print('45-29')
    elif val == 'poor':
        print('36-20')
    elif val == 'very poor':
        print('27-11')
elif predictions[0][4] == 1:
    if val == 'very good':
        print('54-39')
    elif val == 'good':
        print('46-30')
    elif val == 'fair':
        print('37-23')
    elif val == 'poor':
        print('29-15')
    elif val == 'very poor':
        print('20-5')

74-55
